In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from bokeh.io import output_file, output_notebook, show
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
output_notebook()

import warnings
warnings.filterwarnings("ignore")

Loading BokehJS ...

In [2]:
grlist_2 = pd.read_csv("../input/grlist_2.csv")
grlist_s = pd.read_csv("../input/grlist_s.csv")
galist_2 = pd.read_csv("../input/galist_2.csv")
galist_s = pd.read_csv("../input/galist_s.csv")
glist_2 = pd.read_csv("../input/glist_2.csv")
glist_s = pd.read_csv("../input/glist_s.csv")

* grlist_2	-  List of groups and their parameters detected in 2MRS (Table A.1) (31506 rows)
* grlist_s	-  List of groups and their parameters detected in SDSS DR12 (Table A.2) (229893 rows)
* galist_2	-  List of 2MRS galaxies used, as well as their IDs and coordinates (Table A.6) (43425 rows)
* galist_s	-  List of SDSS galaxies used, as well as their IDs and coordinates (Table A.7) (402439 rows)

# Описание колонок

In [3]:
## TODO

In [4]:
_2MRS = galist_2.merge(grlist_2, left_on = 'iGrID', right_on= 'iGrID', how = 'left')
print("2MRS total size:", _2MRS.shape)
_2MRS = _2MRS[_2MRS['Ntot'] > 30]
print("2MRS[Ntot] > 30 size:",_2MRS.shape)

2MRS total size: (43425, 21)
2MRS[Ntot] > 30 size: (1200, 21)


In [5]:
SDSS  = galist_s.merge(grlist_s, left_on = 'iGrID', right_on= 'iGrID', how = 'left')
print("SDSS  total size:", SDSS .shape)
SDSS  = SDSS[SDSS ['Ntot'] > 30]
print("SDSS [Ntot] > 30 size:",SDSS .shape)

SDSS  total size: (402439, 23)
SDSS [Ntot] > 30 size: (30539, 23)


In [6]:
_2MRS['iGrID'].unique()

array([1069,  344,  387,  396,  539,    6,  442,  400,  179,    8, 1007,
        732,   10,   47,  279,  354,  298,  157,   91,  277,  293],
      dtype=int64)

In [7]:
SDSS['iGrID'].unique().tolist()[:10:]

[82182, 2524, 81984, 188046, 86383, 178738, 176410, 188123, 176411, 173489]

In [8]:
def plot_galaxy_group(df, iGrID=1069):
    
    data = df[df['iGrID'] == iGrID]
    print(data.shape)
    source = ColumnDataSource(data)
    source_centre = ColumnDataSource(data.head(1))


    p = figure(plot_width=900, plot_height=450, background_fill_color='#440154', tools='hover')
    p.title.text = 'Galaxy group '+ str(iGrID) + " plot"
    p.grid.visible = True
    p.xaxis.axis_label = 'Right ascension'
    p.yaxis.axis_label = 'Declination'
    
    p.asterisk(x = 'RAJ2000_x', y = 'DEJ2000_x', size= 10, color="yellow",
               alpha = 0.7, source = source)
    p.asterisk(x = 'RAJ2000_y', y = 'DEJ2000_y', size = 20, color = 'yellow', 
               alpha = 1, source = source_centre)
    
    hover = p.select(dict(type=HoverTool))
    hover.tooltips = [
            ("iGalID", "@iGalID"),
            ("Redshift", "@z_x{0.0000}"),
            ("RAJ2000", "$x"),
            ("DEJ2000", "$y"),
    ]
    show(p)

In [16]:
plot_galaxy_group(_2MRS,8)

(205, 21)


#  Расстояния

Вот здесь взял формулу для вычисления: http://spiff.rit.edu/classes/phys373/lectures/radec/radec.html

In [10]:
def angular_distance(ra1, dec1, ra2, dec2):
    return np.sqrt(((ra1-ra2)*np.cos(dec1))**2 + (dec1-dec2)**2)

In [11]:
def plot_scatter(df,iGrID=1069):
    data = df[df['iGrID'] == iGrID]
    data['distance_to_centre'] = data.progress_apply(lambda row: angular_distance(row['RAJ2000_x'], row['DEJ2000_x'],row['RAJ2000_y'], row['DEJ2000_y']), axis=1)
    data['RedShift_diff']= data['z_x']*100 - data['z_y']*100
    source = ColumnDataSource(data)

    p = figure(plot_width=900, plot_height=450, tools='hover')
    p.title.text = 'Scatter for group '+ str(iGrID)
    p.grid.visible = True
    p.xaxis.axis_label = 'distance_to_centre'
    p.yaxis.axis_label = 'RedShift_diff * 100'

    p.circle(x = 'distance_to_centre', y = 'RedShift_diff', size= 10, color="red",
               alpha = 0.7, source = source)

    hover = p.select(dict(type=HoverTool))
    hover.tooltips = [
            ("iGalID", "@iGalID"),
            ("distance_to_centre", "$x"),
            ("RedShift_diff", "$y"),
    ]
    show(p)

In [15]:
plot_scatter(_2MRS, 8)

100%|██████████████████████████████████████████| 205/205 [00:00<00:00, 9514.89it/s]
